In [6]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.cm
import matplotlib.pyplot as plt

In [7]:
wind_angle = np.arctan(4.)
wind_mag = 1.
fly_mag = 1.5
flux_rad = 10.

num_flies = 15000
heading_angles = np.linspace(4*360./num_flies,4*360,num_flies)*np.pi/180

In [8]:
hit_angles = np.arctan((wind_mag*np.sin(wind_angle-heading_angles))/fly_mag)+heading_angles
cmap = matplotlib.cm.get_cmap('bwr')
colors = cmap(np.linspace(1./num_flies,1,num_flies))

In [9]:
from ipywidgets import interactive

In [10]:
def f(wind_mag):
    global wind_angle,fly_mag
    hit_angles = np.arctan((wind_mag*np.sin(wind_angle-heading_angles))/fly_mag)+heading_angles
    plt.figure(1,figsize=(8,8))
    ax=plt.subplot(2,2,1)
    plt.scatter(2*np.cos(heading_angles),2*np.sin(heading_angles),color=colors,alpha=0.01)
    plt.scatter(6*np.cos(hit_angles),6*np.sin(hit_angles),color=colors,alpha=0.01)
    ax.set_aspect('equal', 'datalim')
    ax.set_ylim([-7,7])
    ax = plt.subplot(2,2,4,projection='polar')
    n,bins,_ = plt.hist(hit_angles,bins=500)
    ax.cla()
    plt.plot(bins,np.concatenate((n,[n[0]])))
    ax.set_yticks([])
    ax = plt.subplot(2,2,3)
    plt.plot(heading_angles,hit_angles-heading_angles)
#     ax.set_yticks([])
    ax = plt.subplot(2,2,2,projection='polar')
    plt.scatter(heading_angles,hit_angles-heading_angles,color=colors)
    plt.ylim([0,np.max(hit_angles-heading_angles)])
    ax.set_yticks([])

    plt.show()

interactive_plot = interactive(f, wind_mag=(0, 4.0,0.05))
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=2.0, description='wind_mag', max=4.0, step=0.05), Output(layout=Layout…

In [15]:
def f(w=0.5,theta_d=np.pi/6.):
    n = 10000
    theta_0 = np.arctan(1.)
    r_1 = 1.5
    thetas = np.linspace(360./n,360,n)*np.pi/180
    output = (np.arctan(w*np.sin(theta_0-thetas)/r_1)+thetas)%(2*np.pi)
    plt.figure(1,figsize=(16,8))
    gs = matplotlib.gridspec.GridSpec(2,4)
    plt.subplot(gs[0,0]);plt.title('Final vs. Intended Angle')
    plt.plot(thetas,output,'o',markersize=0.4)
    plt.xlim([0,2*np.pi]);plt.ylim([0,2*np.pi])
    
    w_vec = np.array([w*np.cos(theta_0),w*np.sin(theta_0)])
    r_vec = np.array([r_1*np.cos(theta_d),r_1*np.sin(theta_d)])
    wind_par_vec = (np.dot(w_vec,r_vec))/(r_1**2)*r_vec
    w_perp_vec = w_vec - wind_par_vec
    heading_final_vec = r_vec+w_perp_vec
#     final_angle_demo = np.arctan(heading_final_vec[1]/heading_final_vec[0])
    final_angle_demo = (np.arctan(w*np.sin(theta_0-theta_d)/r_1)+theta_d)%(2*np.pi)
    
    input_demo=plt.plot([theta_d,theta_d],[0,final_angle_demo%(2*np.pi)],'r',label='intended angle')
    output_demo=plt.plot([0,theta_d],[final_angle_demo%(2*np.pi),final_angle_demo%(2*np.pi)],'g',label='final angle',)
    plt.legend(bbox_to_anchor=(0.6,0.9))
    
    plt.subplot(gs[1,0]);plt.title('Derivative')
    deriv = np.diff(output)[np.abs(np.diff(output))<1]
    plt.plot(thetas[:-2],deriv/(np.max(deriv))*2*np.pi,color='b')
    plt.plot(thetas,np.zeros_like(thetas),color='y')
    
    ax=plt.subplot(gs[0,1],projection='polar') 
    n,bins,_ = plt.hist(output%(2*np.pi),bins=500)
    ax.cla()
    plt.plot(bins,np.concatenate((n,[n[0]])))
    ax.set_yticks([])
    _,ymax = ax.get_ylim()
    plt.plot([final_angle_demo,0],[ymax,0],'g');plt.title('Final Angle Histogram',x=0.5,y=1.1)
    
    ax = plt.subplot(gs[1,1])
    plt.plot(bins,np.concatenate((n,[n[0]])))
    ax.set_yticks([])
    plt.xlim([0,2*np.pi])
    _,ymax = ax.get_ylim()
    
    plt.plot([final_angle_demo,final_angle_demo],[0,ymax],'g',label='final angle')
    plt.legend(bbox_to_anchor=(0.9,1.12))
    
    
    plt.subplot(gs[:,2:4]);plt.title('Vector Summation for Theta_d')
    original_heading = plt.arrow(0,0,r_1*np.cos(theta_d),r_1*np.sin(theta_d), head_width=0.05, head_length=0.1, fc='r', ec='r',
                                length_includes_head=True)
   
    wind_vector = plt.arrow(*r_vec,*w_vec,
                            head_width=0.05, head_length=0.1, fc='b', ec='b',length_includes_head=True)
#     wind_par = plt.arrow(*r_vec,*wind_par_vec, 
#                             head_width=0.05, head_length=0.1, fc='orange', ec='orange')
    wind_perp = plt.arrow(*r_vec,*w_perp_vec,
                            head_width=0.05, head_length=0.1, fc='dodgerblue', ec='dodgerblue',length_includes_head=True)
    wind_par = plt.arrow(*r_vec,*wind_par_vec,
                            head_width=0.05, head_length=0.1, fc='dodgerblue', ec='dodgerblue',length_includes_head=True)
    heading_final = plt.arrow(0,0,*heading_final_vec,
                             head_width=0.05, head_length=0.1, fc='g', ec='g',length_includes_head=True)
    
    plt.ylim([-2,3]);plt.xlim([-2,3])
    plt.legend([original_heading,wind_vector,wind_perp,wind_par,heading_final],
                   ['intended velocity','wind vector','wind perp','wind par','final velocity'],
                    bbox_to_anchor=(1.4,0.5))
    
    plt.show()
interactive_plot = interactive(f, w=(0, 4.0,0.05),theta_d=(0,2*np.pi,0.05))
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=0.5, description='w', max=4.0, step=0.05), FloatSlider(value=0.5235987…